# Pride & Prejudice

In this notebook, we'll build a character-wise RNN trained on Wings of Fire, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

In [0]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [0]:
with open('Pride_and_Prejudice.txt', encoding="utf8", errors='ignore') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy.

In [3]:
text[:100]

'Chapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune,'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([24, 53, 46, 61, 65, 50, 63,  1, 10,  0, 30, 65,  1, 54, 64,  1, 46,
        1, 65, 63, 66, 65, 53,  1, 66, 59, 54, 67, 50, 63, 64, 46, 57, 57,
       70,  1, 46, 48, 56, 59, 60, 68, 57, 50, 49, 52, 50, 49,  6,  1, 65,
       53, 46, 65,  1, 46,  1, 64, 54, 59, 52, 57, 50,  1, 58, 46, 59,  1,
       54, 59,  1, 61, 60, 64, 64, 50, 64, 64, 54, 60, 59,  1, 60, 51,  1,
       46,  1, 52, 60, 60, 49,  1, 51, 60, 63, 65, 66, 59, 50,  6],
      dtype=int32)

In [5]:
encoded.shape

(679439,)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [6]:
len(vocab)

72

## Making training mini-batches

In [0]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size*n_steps
    n_batches = len(arr) // (characters_per_batch)
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches*characters_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size,-1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y_temp = arr[:, n+1:n+n_steps+1]
        
        # For the very last batch, y will be one character short at the end of 
        # the sequences which breaks things. To get around this, I'll make an 
        # array of the appropriate size first, of all zeros, then add the targets.
        # This will introduce a small artifact in the last batch, but it won't matter.
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [0]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [9]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[24 53 46 61 65 50 63  1 10  0]
 [46 65  1 53 50 63  1 51 60 63]
 [ 1 54 59  1 65 53 50  1 61 63]
 [ 1 49 50 65 50 63 58 54 59 50]
 [ 1 65 53 50 63 50  1 68 50 63]
 [51 63 60 58  1 70 60 66 63  1]
 [52 53 65 50 59 50 49  1 48 60]
 [ 1 47 50 57 54 50 67 50  1 65]
 [50 46 52 50 63 59 50 64 64  1]
 [60 66 64  1 46 59 49  1 51 60]]

y
 [[53 46 61 65 50 63  1 10  0 30]
 [65  1 53 50 63  1 51 60 63  1]
 [54 59  1 65 53 50  1 61 63 60]
 [49 50 65 50 63 58 54 59 50 49]
 [65 53 50 63 50  1 68 50 63 50]
 [63 60 58  1 70 60 66 63  1 64]
 [53 65 50 59 50 49  1 48 60 58]
 [47 50 57 54 50 67 50  1 65 53]
 [46 52 50 63 59 50 64 64  1 65]
 [66 64  1 46 59 49  1 51 60 63]]


## Building the model

### Inputs

In [0]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape=(batch_size,num_steps),name='inputs')
    targets = tf.placeholder(tf.int32, shape=(batch_size,num_steps),name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [0]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

In [0]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output,axis = 1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output,[-1,in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size,out_size),stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x,softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits=logits,name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. 

In [0]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets,num_classes)
    y_reshaped = tf.reshape(y_one_hot,logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y_reshaped)
    loss=tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [0]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate,epsilon = 0.001)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [0]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size,num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs,num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state =tf.nn.dynamic_rnn(cell, x_one_hot,initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

In [0]:
batch_size = 32        # Sequences per batch
num_steps = 50         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

In [17]:
epochs = 20
# Print losses every N interations
print_every_n = 200

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
                '''
                if(e > 0): 
                  change = np.abs(temp-batch_loss)
                  learning_rate = learning_rate * change  # multiply by the relative change
                temp=batch_loss
                '''
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 200...  Training loss: 2.8508...  0.1493 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.3665...  0.1486 sec/batch
Epoch: 2/20...  Training Step: 600...  Training loss: 2.1674...  0.1478 sec/batch
Epoch: 2/20...  Training Step: 800...  Training loss: 2.0302...  0.1489 sec/batch
Epoch: 3/20...  Training Step: 1000...  Training loss: 1.8547...  0.1533 sec/batch
Epoch: 3/20...  Training Step: 1200...  Training loss: 1.7492...  0.1556 sec/batch
Epoch: 4/20...  Training Step: 1400...  Training loss: 1.6297...  0.1537 sec/batch
Epoch: 4/20...  Training Step: 1600...  Training loss: 1.5259...  0.1552 sec/batch
Epoch: 5/20...  Training Step: 1800...  Training loss: 1.5168...  0.1541 sec/batch
Epoch: 5/20...  Training Step: 2000...  Training loss: 1.4657...  0.1506 sec/batch
Epoch: 6/20...  Training Step: 2200...  Training loss: 1.4808...  0.1503 sec/batch
Epoch: 6/20...  Training Step: 2400...  Training loss: 1.3954...  0.1539 sec/batch
Epoch: 7

#### Saved checkpoints

In [18]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i8480_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l512.ckpt"
all_model_checkpoint_pa

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.


In [0]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [0]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [21]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i8480_l512.ckpt'

In [22]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 5000, lstm_size, len(vocab), prime="Dream")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i8480_l512.ckpt
Dream heart that such an hour were at all. And the servants to have having seen the present, how said they would have been so fortunately in the repeated confusion. At the same period of their name than in the room.

Elizabeth was a visit at Longbourn, and a folly and admiration of his attachment, as she would now say the settled and countenance of them there was a story of sink at home.

In addre side of the sense, so thought at such a consolation of saying:

In a companion as they have been any admiration of the first consceart, and said his form of her sister, but with as the pleasure of her party, and, was she said:

In daughters and the case. I shall be as few of it, and was this is a very longer side at home. He seemed to say that her list is so disposed to means in the match in town.

The address of their constant parties were talking of that the same time she would say shall as to see the house, as they warme

In [23]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Dream")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Dreame,, what she sise ant her so han tuers of a dist of hin wish thite asd bis an the hir anthir and whe thith the tilt the with that the wor hous tourd to hir thougt too the ang ot to assingers, ber te his te ther wes at as tho mint tho him has tonter on wathe whith hever sust bunt alt thet the the soun wo hor sis the sand to her a salling the has silly of whe shim ald an thes the wis of theng hot, hed she torle als bitt he sore the the heve andentite of andere fom the mish, withe ture so cantire.

The sas a deatt the her sotine what and alt ald int on wether wast at in wimen ast and of who want att ance were that, be ather se songer at thete ande went the se tion his thingess to ald ald him thit thit of the hen sersile be soture thas and ther the har ther heve sount hor wead ande whet tull alt te tho sher wallyy of aredirene tho heve ther wes th mimentherser of whourd and tho gone who har avy him te wimh, had we ha

In [24]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Dream")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Dreaming their sine ot there that the could so bithing to should, and soun the saint his asser, and should a converysan enough to thouke a sto to be the sure of had but well tatious of the rost a more at her so till. The felforedy the radion. There sour in secirest..

I was nover hound. There firsition that he hereffrended to the plakence. The fast she side of the sempartion to ang any to discalfest her bettinges. Tree tality a to befor her than stine, and with the countionss and has ploation to bothises, shis wele a dore of her seece, whon ant the supting and the camone of that shall the carest, and as siscantating and any siming when then a mintlons, whon who will take to him a fithire as suried hor. Bitken tame house a mutter it, her was so mad had sood ald and in thay. Who shis had the palies of than a much of she sat then so hap his suppess at a many his presend that such and and thoul the fournt on sisceries of

In [25]:
checkpoint = 'checkpoints/i2400_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Dream")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i2400_l512.ckpt
Dreamance the particular to sain herselvess which had all the sume together, that it, and asserable tolteme without interest, that is whether see of such and other only that is wonder, too marry, and a formones were receiving in the rang at her as an all the resign to be a supper. Her fame that I have been, sointance, and it will so make her; and had the sabserve to have the stard, and the sepone were answer, and as surpecsion and trought and the sure in she had to have been a mind by the person that you certainly a sawe most into the saming too most all the conversation with his farious, see where he was so send an agiment, that he could have been mentioned in the care with mose which a said with the whole. He had allowed to the person of my probesce immediately to the who as made it, she ad some offerense of a certain were seet to thanks and the subject and towarss in superitious in her for a choose. They will not 

In [26]:
checkpoint = 'checkpoints/i4800_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Dream")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i4800_l512.ckpt
Dreamon attentions it went as to see him to be so much or herself, and as he sais so to say to my dear.

Mary had seen the life of the morning to a conferemation at talking at he wanted to her at hamber we shall not be askuned on him.

My dear, since, said Elizabeth, then she said to ters it is in my father will, and seeing him; and to talk to the motive, and all half have the country, but it is not as he should be disagreeable to think what any of any of its answer, but his character are danger of him which she could not be discovered, and to the streater there was nothing at her of him, and were now interested by some occasion, who was tenting the matter of having at earnest, was shortly as there was a country this meaning in her father, to be dared before they were oft no longer. The character was expected and alsure; and she wish to see what I had the person will the part of her time, and seems without it. I am a